In [ ]:
import cvxpy as cp
import numpy as np
import matplotlib.pyplot as plt
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import math
import pandas as pd
import pickle
torch.autograd.set_detect_anomaly(True)

In [ ]:
N = 2
try:
    with open(str(N)+'_user_dic.pkl', 'rb') as f:
        loaded_dict = pickle.load(f)
        print("loaded saved results")
except:
    print("result unfound")

In [ ]:
alpha_params = np.random.rand(N)
beta_params = np.random.rand(N)*100

def quadatic_util(x_var, alpha_params, beta_params, p):
    """
    return surplus of the system
    """
    w_lst = []
    for i in range(len(alpha_params)):
        w_lst.append(-alpha_params[i]*x_var[i]**2 + beta_params[i]*x_var[i] - p*x_var[i])
    return w_lst

l_range = np.arange(0.1, 100, 0.1)

In [ ]:
# grid search is only applicable to 2-agents system, and is mainly used for sanity check
if N == 2:
    feasible_w = []
    feasible_allocation = []
    NBS_value = -1e5
    SW_value = -1e5
    NBS_point = None
    SW_point = None
    max_min_point = None
    max_min_value = -1e5
    NBS_allocation = None
    SW_allocation = None
    Maxmin_allocation = None
    for L in l_range:
        for x1 in np.arange(0, L, 0.005):
            x2 = L - x1
            p = 2*L
            w1, w2 = quadatic_util([x1, x2], alpha_params, beta_params, p)
    #         = -alpha_params[0]*x1**2 + beta_params[0]*x1-p*x1
    #         w2 = -alpha_params[1]*x2**2 + beta_params[0]*x2-p*x2
            if w1 > 0 and w2 > 0 and x1 >= 0 and x2 >= 0:
                feasible_w.append([w1, w2])
                feasible_allocation.append([x1, x2])
                if w1*w2 > NBS_value:
                    NBS_point = (w1, w2)
                    NBS_value = w1*w2
                    NBS_allocation = (x1, x2)
                if w1+w2 > SW_value:
                    SW_point = (w1, w2)
                    SW_value = w1+w2
                    SW_allocation = (x1, x2)
                if min(w1, w2) > max_min_value:
                    max_min_value = min(w1, w2)
                    max_min_point = (w1, w2)
                    Maxmin_allocation = (x1, x2)
    feasible_w = np.array(feasible_w)
    feasible_allocation = np.array(feasible_allocation)

In [ ]:
try:
    plt.scatter(feasible_w[:, 0], feasible_w[:, 1])
    plt.scatter(NBS_point[0], NBS_point[1], label="Nash Bargaining")
    plt.scatter(SW_point[0], SW_point[1], label="Social Welfare")
    plt.scatter(max_min_point[0], max_min_point[1], label="Max-min")
    plt.xlabel("surplus of player 1")
    plt.ylabel("surplus of player 2")
    plt.legend()
    plt.title("feasible region of (s1, s2)")
except: 
    pass

In [ ]:
SW_result_lst = []
h = 0
for L in l_range:
    if h%10 == 0:
        print("=", end='')
    h += 1
    p = 2*L
    x_sw = cp.Variable(N)
    w_lst = quadatic_util(x_sw, alpha_params, beta_params, p)
    objective_sw = cp.Maximize(cp.sum(w_lst))
    constraints_sw = [x_sw >=0, sum(x_sw) == L]
    for j in range(N):
        constraints_sw.append(w_lst[j] >= 0)
#     constraints_sw = [x_sw >=0, -alpha1*x_sw[0] + beta1-p >=0, -alpha2*x_sw[1] + beta2-p>=0, sum(x_sw) == L]
    prob_sw = cp.Problem(objective_sw, constraints_sw)
    result = prob_sw.solve()
    SW_optimizer = x_sw.value
    if objective_sw.value == None or math.isnan(objective_sw.value):
        SW_result_lst.append(-1e4)
    else:
        SW_result_lst.append(objective_sw.value)

sw_opt_L = l_range[np.argmax(SW_result_lst)]
sw_opt_p = sw_opt_L * 2

x_sw = cp.Variable(N)
w_lst = quadatic_util(x_sw, alpha_params, beta_params, sw_opt_p)
objective_sw = cp.Maximize(cp.sum(w_lst))
constraints_sw = [x_sw >=0, sum(x_sw) == sw_opt_L]
for j in range(N):
    constraints_sw.append(w_lst[j] >= 0)

prob_sw = cp.Problem(objective_sw, constraints_sw)
result = prob_sw.solve()
SW_optimizer = x_sw.value
CE_SW = [wi.value for wi in w_lst]

In [ ]:
PF_result_lst = []
h = 0
for L in l_range:
    if h%10 == 0:
        print("=", end='')
    h += 1
    p = 2*L
    x_pf = cp.Variable(N)
    w_lst = quadatic_util(x_pf, alpha_params, beta_params, p)
    objective_pf = cp.Maximize(cp.sum([cp.log(wi) for wi in w_lst]))
    constraints_pf = [x_pf >=0,sum(x_pf) == L]
    for j in range(N):
        constraints_pf.append(w_lst[j] >= 0)
    prob_pf = cp.Problem(objective_pf, constraints_pf)
    
    try:
        result = prob_pf.solve()
    except:
        result = prob_pf.solve(solver=cp.SCS)

    if objective_pf.value == None or math.isnan(objective_pf.value):
        PF_result_lst.append(-1e4)
    else:
        PF_result_lst.append(objective_pf.value)

pf_opt_L = l_range[np.argmax(PF_result_lst)]
pf_opt_p = pf_opt_L*2

x_pf = cp.Variable(N)
w_lst = quadatic_util(x_pf, alpha_params, beta_params, pf_opt_p)
objective_pf = cp.Maximize(cp.sum([cp.log(wi) for wi in w_lst]))
constraints_pf = [x_pf >=0, sum(x_pf) == pf_opt_L]
for j in range(N):
    constraints_pf.append(w_lst[j] >= 0)
prob_pf = cp.Problem(objective_pf, constraints_pf)
try:
    result = prob_pf.solve()
except:
    result = prob_pf.solve(solver=cp.SCS)


pf_optimizer = x_pf.value
PF_optimizer_surplus = [wi.value for wi in w_lst]

In [ ]:
maxmin_result_lst = []
h = 0
for L in l_range:
    if h%10 == 0:
        print("=", end='')
    h += 1
    p = 2*L
    x_mm = cp.Variable(N)
    w_lst = quadatic_util(x_mm, alpha_params, beta_params, p)
    objective_mm = cp.Maximize(cp.min(cp.vstack(w_lst)))
    constraints_mm = [x_mm >=0, sum(x_mm) == L]
    for j in range(N):
        constraints_mm.append(w_lst[j] >= 0)
    prob_mm = cp.Problem(objective_mm, constraints_mm)
    result = prob_mm.solve()
    if objective_mm.value == None or math.isnan(objective_mm.value):
        maxmin_result_lst.append(-1e4)
    else:
        maxmin_result_lst.append(objective_mm.value)

mm_opt_L = l_range[np.argmax(maxmin_result_lst)]
mm_opt_p = mm_opt_L*2

x_mm = cp.Variable(N)
w_lst = quadatic_util(x_mm, alpha_params, beta_params, mm_opt_p)
objective_mm = cp.Maximize(cp.min(cp.vstack(w_lst)))
constraints_mm = [x_mm >=0, sum(x_mm) == mm_opt_L]
for j in range(N):
    constraints_mm.append(w_lst[j] >= 0)
prob_mm = cp.Problem(objective_mm, constraints_mm)
result = prob_mm.solve()

Maxmin_optimizer = x_mm.value
MM_optimizer_surplus = [wi.value for wi in w_lst]

In [ ]:
ALPHA_fairness_dic = {}
for ALPHA in [0.05, 0.1, 0.5, 1.5, 5]:
    print("*", end='')
    ALPHA_result_lst = []
    h = 0
    for L in l_range:
        if h%10 == 0:
            print("=", end='')
        h += 1
        p = 2*L
        x_alpha = cp.Variable(N)
        w_lst = quadatic_util(x_alpha, alpha_params, beta_params, p)
        objective_alpha = cp.Maximize(cp.sum([cp.power(wi, 1-ALPHA) for wi in w_lst])/(1-ALPHA))
        constraints_alpha = [x_alpha >=0, sum(x_alpha) == L]
        for j in range(N):
            constraints_alpha.append(w_lst[j] >= 0)
        prob_alpha = cp.Problem(objective_alpha, constraints_alpha)
        try:
            result = prob_alpha.solve()
        except:
            result = prob_alpha.solve(solver=cp.SCS)
        
        if objective_alpha.value == None or math.isnan(objective_alpha.value):
            ALPHA_result_lst.append(-1e4)
        else:
            ALPHA_result_lst.append(objective_alpha.value)
    alpha_opt_L = l_range[np.argmax(ALPHA_result_lst)]
    alpha_opt_p = alpha_opt_L * 2
    
    x_alpha = cp.Variable(N)
    w_lst = quadatic_util(x_alpha, alpha_params, beta_params, alpha_opt_p)
    objective_alpha = cp.Maximize(cp.sum([cp.power(wi, 1-ALPHA) for wi in w_lst])/(1-ALPHA))
    constraints_alpha = [x_alpha >=0, sum(x_alpha) == alpha_opt_L]
    for j in range(N):
        constraints_alpha.append(w_lst[j] >= 0)
    prob_alpha = cp.Problem(objective_alpha, constraints_alpha)
    try:
        result = prob_alpha.solve()
    except:
        result = prob_alpha.solve(solver=cp.SCS)
    ALPHA_fairness_dic[ALPHA] = [wi.value for wi in w_lst]

In [ ]:
#organize the results in dataframe
alpha_fairness_pd = pd.DataFrame.from_dict(ALPHA_fairness_dic).T
alpha_fairness_pd = alpha_fairness_pd.sort_index()

In [ ]:
alpha_fairness_pd

In [ ]:
x_pf = cp.Variable(N)
w_lst = quadatic_util(x_pf, alpha_params, beta_params, pf_opt_p)
objective_pf = cp.Maximize(cp.sum([cp.log(wi) for wi in w_lst]))
constraints_pf = [x_pf >=0, sum(x_pf) == pf_opt_L]
for j in range(N):
    constraints_pf.append(w_lst[j] >= 0)
prob_pf = cp.Problem(objective_pf, constraints_pf)
try:
    result = prob_pf.solve()
except:
    result = prob_pf.solve(solver=cp.SCS)


pf_optimizer = x_pf.value
PF_optimizer_surplus = [wi.value for wi in w_lst]

In [ ]:
x_pf = cp.Variable(N)
w_lst = quadatic_util(x_pf, alpha_params, beta_params, pf_opt_p)
objective_pf = cp.Maximize(cp.sum([cp.log(wi) for wi in w_lst]))
constraints_pf = [x_pf >=0, sum(x_pf) == pf_opt_L]
for j in range(N):
    constraints_pf.append(w_lst[j] >= 0)
prob_pf = cp.Problem(objective_pf, constraints_pf)
try:
    result = prob_pf.solve()
except:
    result = prob_pf.solve(solver=cp.SCS)


pf_optimizer = x_pf.value
PF_optimizer_surplus = [wi.value for wi in w_lst]

### fairness-efficiency trade-off

In [ ]:
for key, value in ALPHA_fairness_dic.items():
    if key in [0.05, 0.1, 0.5, 1.5]:
        y_point = min(value)
        x_point = (sum(value))
        plt.scatter(x_point, y_point, label=str(key))
plt.scatter(sum(MM_optimizer_surplus), min(MM_optimizer_surplus), label='max-min')
plt.scatter(sum(CE_SW), min(CE_SW), label='social welfare')
plt.scatter(sum(PF_optimizer_surplus), min(PF_optimizer_surplus), label='PF')
# plt.scatter(sum(NBS_point), min(NBS_point), label='PF')
plt.ylabel("fairness (minimum surplus)")
plt.xlabel("utilitarian (social welfare)")
plt.title("fairness trade-off")
plt.legend()

In [ ]:
#price of fairness evaluation
SYSTEM = sum(CE_SW)
maxmin_FAIR = sum(MM_optimizer_surplus)
# pf_FAIR = sum(PF_optimizer_surplus)
pf_FAIR = sum(PF_optimizer_surplus)

In [ ]:
bar_lst = [0]
bar_label_lst=["SW"]
for key in [0.05, 0.1, 0.5]:
    value = ALPHA_fairness_dic[key]
    bar_label_lst.append(str(key))
    alpha_FAIR = sum(value)
    bar_lst.append((SYSTEM-alpha_FAIR)/SYSTEM)
bar_lst.append((SYSTEM - pf_FAIR)/SYSTEM) 
bar_label_lst.append("PF")
for key, value in ALPHA_fairness_dic.items():
    if key in [1.5]:
        bar_label_lst.append(str(key))
        alpha_FAIR = sum(value)
        bar_lst.append((SYSTEM-alpha_FAIR)/SYSTEM)

bar_label_lst.append("MaxMin")
bar_lst.append((SYSTEM - maxmin_FAIR)/SYSTEM)

In [ ]:
plt.bar(bar_label_lst, bar_lst)
plt.title("Price of Fairness")
plt.ylabel("PoF")

In [ ]:
plt.scatter(N, (SYSTEM - maxmin_FAIR)/SYSTEM, label='maxmin')
plt.scatter(N, (SYSTEM - pf_FAIR)/SYSTEM, label='proportional fairness')
# plt.scatter(2, 0, label="social welfare")
for key, value in ALPHA_fairness_dic.items():
    if key in [0.05, 0.1, 0.5, 1.5, 5]:
        alpha_FAIR = sum(value)
        plt.scatter(N, (SYSTEM-alpha_FAIR)/SYSTEM, label="alpha="+str(key))
plt.legend()
plt.xlabel("system size")
plt.ylabel("Price of Fairness")
plt.title("price of fairness")

In [ ]:
ALPHA_fairness_dic['SW'] = CE_SW
ALPHA_fairness_dic['MAXMIN'] = MM_optimizer_surplus
ALPHA_fairness_dic['PF'] = PF_optimizer_surplus

#save results along the way
with open(str(N)+'_user_dic.pkl', 'wb') as f:
    pickle.dump(ALPHA_fairness_dic, f)